In [1]:
from netCDF4 import Dataset
import numpy as np
import glob
import os
import pandas as pd
import wrf

import cartopy.crs as ccrs
from cartopy.io.shapereader import Reader
import cartopy.feature as cfeature
from cartopy import config
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.feature import ShapelyFeature

from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
from matplotlib.image import imread

In [4]:
wrfile_d02 = Dataset('/media/lucas/ce7c746b-5271-4ebe-a443-a00fe8fb43f4/wrf/WRF/run/wrfout_d02_2018-05-22_18:00:00')
torres = pd.read_csv('/home/lucas/dados_vento/lat_lon_towers.txt', usecols=[1,2]).values
topo = Dataset('/media/lucas/ce7c746b-5271-4ebe-a443-a00fe8fb43f4/wrf/WRF/run/geo_em.d02.nc')

In [5]:
u10 = wrf.getvar(wrfile_d02, 'U10')
v10 = wrf.getvar(wrfile_d02, 'V10')
wind10 = np.sqrt(u10*u10+v10*v10)

In [6]:
wspeed = wrf.getvar(wrfile_d02,'wspd_wdir')[0]
wdir = wrf.getvar(wrfile_d02,'wspd_wdir')[1]
z = wrf.getvar(wrfile_d02, 'z', units='m')
slp = wrf.getvar(wrfile_d02,'slp')
p = wrf.getvar(wrfile_d02,'pressure')
lon = wrf.getvar(wrfile_d02,'lon')
lat = wrf.getvar(wrfile_d02,'lat')
t2 = wrf.getvar(wrfile_d02,'T2')
geo = wrf.getvar(topo, 'HGT_M')

In [7]:
a = wrf.interplevel(wspeed, z, [312.43304+108,336.6772+108,298.94324+108])

In [8]:
a = wrf.interplevel(wspeed, z, [312.43304+108,336.6772+108,298.94324+108])
n=1
coordenada=[]
wind=[]
topografia=[]
while n<4:
    abslat = np.abs(lat-torres[n,0])
    abslon = np.abs(lon-torres[n,1])
    c = np.maximum(abslon,abslat)
    x, y = np.where(c == np.min(c))
    wind_ = a[1,x,y].values
    topografia_ = geo[x,y]
    wind.append(wind_)
    topografia.append(topografia_)
    n+=1

In [9]:
wind[0]

array([[4.5143714]], dtype=float32)

In [15]:
wind_obs

4.08

In [11]:
indx = pd.to_datetime(pd.read_csv('ecc1_.csv').data.values)

In [12]:
arquivo = pd.read_csv('ecc1_.csv')

In [13]:
arquivo.index = indx

In [14]:
wind_obs = float(arquivo.vs_med.loc['2018-05-23 06:00:00'])

In [16]:
def f(x):
    return float(x)

In [17]:
f2 = np.vectorize(f)

In [18]:
topografia = f2(topografia)

In [19]:
topografia.mean()

318.06292724609375

In [82]:
del(torres['Unnamed: 0'])

In [23]:
round(float(wind[0].values),3), round(float(wind[1].values),3), round(float(wind[2].values),3)

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [21]:
df = pd.read_csv('ecc1_.csv')
df2 = pd.read_csv('ecc2_.csv')
df3 = pd.read_csv('ecc3_.csv')

In [22]:
float(df[df.data=='2018-05-23 00:00:00']['vi_max']), \
float(df2[df2.data=='2018-05-23 00:00:00']['vi_max']), \
float(df3[df3.data=='2018-05-23 00:00:00']['vi_max'])

(5.17, 4.66, 4.94)

In [24]:
df

,data,hora,erro,hPa,tp,ur,vs_med,vs_max,vs_min,vs_dev,...,vi_med,vi_max,vi_min,vi_dev,di_med,di_dev,vm_med,vm_max,vm_min,vm_dev
0,2018-05-23 00:00:00,0,0,980,11.6,61.5,4.5,4.89,4.24,0.17,...,4.86,5.17,4.57,0.14,187,1,4.53,4.95,4.21,0.17
1,2018-05-23 00:10:00,1000,0,980,11.6,61.5,5.12,5.39,4.75,0.12,...,5.17,5.49,4.94,0.09,182,1,4.46,4.99,4.12,0.22
2,2018-05-23 00:20:00,2000,0,980,11.6,60.8,5.2,5.44,5.03,0.06,...,5.28,5.44,5.03,0.08,182,0,4.4,4.86,4.21,0.1
3,2018-05-23 00:30:00,3000,0,980,11.6,60.8,5.04,5.21,4.84,0.06,...,4.93,5.26,4.71,0.14,183,0,4.79,5.13,4.44,0.16
4,2018-05-23 00:40:00,4000,0,980,11.6,59.2,4.92,5.12,4.75,0.06,...,4.78,4.94,4.62,0.06,184,1,4.96,5.13,4.81,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1579,2018-12-24 23:10:00,231000,0,974,14.1,69.7,12.36,16.19,8.3,1.33,...,10.06,14.5,6.14,1.54,158,6,9.5,15.27,5.78,1.84
1580,2018-12-24 23:20:00,232000,0,974,14.1,64.7,16.57,20.21,10.05,1.68,...,13.1,18.18,6.96,1.82,158,5,12.04,16.98,7.85,1.72
1581,2018-12-24 23:30:00,233000,0,974,13.9,61.6,17.63,19.93,14.9,1.03,...,14.65,18.46,9.81,1.54,160,4,12.91,18.09,9.6,1.62
1582,2018-12-24 23:40:00,234000,0,975,13.6,61.3,14.49,18.41,11.12,1.55,...,11.77,16.11,8.02,1.67,157,4,10.82,17.39,7.25,2
